In [1]:
import pandas as pd
import numpy as np
import random
from tqdm.auto import tqdm
from tqdm.contrib import tzip
from sklearn.model_selection import train_test_split
from catboost import CatBoostRanker,Pool,cv,CatBoostRegressor, CatBoostClassifier
import gc

c:\Users\kostopr4v\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df = pd.read_csv('train.csv',parse_dates=['timestamp'])
user_data = pd.read_csv('user.csv')
video_data = pd.read_csv('video.csv')
owner_data = pd.read_csv('owner.csv')

C:\Users\kostopr4v\AppData\Local\Temp\ipykernel_13548\1568392045.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_df = pd.read_csv('train.csv',parse_dates=['timestamp'])


In [3]:
val_predicts_tf = pd.read_parquet("TFIDF_preds_val100.parquet")
val_predicts_als = pd.read_parquet("ALS_preds_val50.parquet")

In [4]:
def chrono_split(
        df: pd.DataFrame, 
        split_by_column: str = 'user_id', 
        ratio: float = 0.7, 
        col_timestamp: str = 'timestamp'):

    df = df.sort_values([split_by_column, col_timestamp])
    groups = df.groupby(split_by_column)

    df["count"] = groups[split_by_column].transform("count")
    df["rank_s"] = groups.cumcount() + 1

    ratio = [ratio, 1 - ratio]
    splits = []
    prev_threshold = None
    for threshold in np.cumsum(ratio):
        condition = df["rank_s"] <= round(threshold * df["count"])
        if prev_threshold is not None:
            condition &= df["rank_s"] > round(prev_threshold * df["count"])
        splits.append(df[condition].drop(["rank_s", "count"], axis=1))
        prev_threshold = threshold

    return splits

def train_val_split(
        train_df: pd.DataFrame, 
        val_users_n: int = 200_000):
    
    user_ids = train_df['user_id'].unique()
    user_ids_val = random.sample(list(user_ids), val_users_n)
    condition = train_df['user_id'].isin(user_ids_val)

    val = train_df[condition]
    val_no_targets, val_targets = chrono_split(val, ratio=0.7)

    train = pd.concat([train_df[~condition], val_no_targets]).sort_values('timestamp')
    return train, val_no_targets, val_targets

In [5]:
random.seed(56)
train, val_no_targets, val_targets = train_val_split(train_df,val_users_n=100_000)

In [6]:
val_group = val_targets.groupby('user_id')
act = val_group['video_id'].agg(lambda x:x.tolist()).tolist()
val_users = val_group.agg(lambda x:x.tolist()).index.tolist()
val_predicts_tf['user_id'] = val_users
val_predicts_tf['rank_i'] = val_predicts_tf['preds'].map(lambda x: list(range(len(x))))
val_predicts_als['user_id'] = val_users
val_predicts_als['rank_i'] = val_predicts_als['preds'].map(lambda x: list(range(len(x))))

In [7]:
def create_data(predicts):
    ranks = []
    video_id = []
    user_id = []
    rank_id = []
    for idx in tqdm(predicts.index):
        id = predicts.loc[idx]['user_id']
        for pred,rank,r_id in zip(predicts['preds'][idx],predicts['scores'][idx],predicts['rank_i'][idx]):
            ranks.append(rank)
            video_id.append(pred)
            user_id.append(id)
            rank_id.append(r_id)
    
    df = pd.DataFrame()
    df['user_id'] = user_id
    df['ranks'] = ranks
    df['rank_id'] = rank_id
    df['video_id'] = video_id
    
    return df

In [8]:
df_tf = create_data(val_predicts_tf)

100%|██████████| 100000/100000 [00:06<00:00, 14735.96it/s]


In [9]:
df_als = create_data(val_predicts_als)

100%|██████████| 100000/100000 [00:06<00:00, 15004.05it/s]


In [10]:
df_tf = df_tf.rename({'ranks':'ranks_tf','rank_id':'rank_id_tf'},axis=1)
df_als = df_als.rename({'ranks':'ranks_als','rank_id':'rank_id_als'},axis=1)

In [11]:
tqdm.pandas()
df_tf['user_id_per_video_id'] = df_tf.progress_apply(lambda x: str(x.video_id) + '_' + str(x.user_id),axis=1)
df_als['user_id_per_video_id'] = df_als.progress_apply(lambda x: str(x.video_id) + '_' + str(x.user_id),axis=1)

100%|██████████| 10000000/10000000 [02:16<00:00, 73052.79it/s]


In [12]:
df_als = df_als.set_index('user_id_per_video_id')
df_tf = df_tf.set_index('user_id_per_video_id')

In [13]:
df_all = pd.concat([df_als,df_tf],axis=1)

In [14]:
del df_als
del df_tf

In [15]:
df_all['user_id'] = df_all['user_id'].fillna(-1)
users_id = np.max(df_all['user_id'].values,axis=-1)

In [16]:
df_all['video_id'] = df_all['video_id'].fillna(-1)
video_id = np.max(df_all['video_id'].values,axis=-1)

In [17]:
df_all = df_all.drop(['user_id','video_id'],axis=1)
df_all['video_id'] = video_id

In [18]:
df_all['user_id'] = users_id
df_all['rank_id_als'] = df_all['rank_id_als'].fillna(500)
df_all['rank_id_tf'] = df_all['rank_id_tf'].fillna(500)
df_all['ranks_als'] = df_all['ranks_als'].fillna(-1)
df_all['ranks_tf'] = df_all['ranks_tf'].fillna(-1)
df = df_all

In [19]:
df['rank_id_als'] = df['rank_id_als'].fillna(500)
df['rank_id_tf'] = df['rank_id_tf'].fillna(500)
df['ranks_als'] = df['ranks_als'].fillna(-1)
df['ranks_tf'] = df['ranks_tf'].fillna(-1)

In [20]:
df.to_parquet('DF_TEST.parquet.gzip',compression='gzip')

In [21]:
val_targets['video_id_liked'] = val_targets.apply(lambda x: x.video_id if x.interaction_type == 'like' else -1,axis=1)
val_targets_base = val_targets.groupby('user_id')['video_id'].agg(lambda x:x.tolist())
val_targets_like = val_targets.groupby('user_id')['video_id_liked'].agg(lambda x:x.tolist())
val_targets_like = val_targets_like.map(set)
val_targets_base = val_targets_base.map(set)
def set_label(df,val_targets_like,val_targets_base):
    labels = []
    for user,item in tzip(df['user_id'],df['video_id']):
        if item in val_targets_like[user]:
            labels += [1]
        elif item in val_targets_base[user]:
            labels += [0.1]
        else:
            labels += [0]
    df['label'] = labels
    return df
df = set_label(df,val_targets_like,val_targets_base)

100%|██████████| 15764694/15764694 [01:28<00:00, 178920.52it/s]


In [ ]:
df = df.reset_index()
df.to_parquet('DF_TEST.parquet.gzip',compression='gzip')

In [7]:
df = pd.read_parquet('DF_TEST.parquet.gzip')

In [8]:
def get_video_like_watching_features(df):
    df['is_like'] = df['interaction_type'].map(lambda x:1 if x == 'like' else 0)
    features = df.groupby('video_id')['is_like'].agg(['sum','mean','count'])
    #features['video_mean_age'] = df.groupby('video_id')['age'].agg(['mean'])
    #features['video_std_age'] = df.groupby('video_id')['age'].agg(['std'])
    features['_'] = range(features.shape[0])
    features['video_id'] = features.index
    features = features.set_index('_')
    return features

video_like_faeatures = get_video_like_watching_features(train)
video_data = video_data.merge(owner_data)

In [9]:
video_data = video_data.merge(video_like_faeatures)

In [10]:
def generate_video_owner_features(video_data):
    video_data['owner_count'] = video_data.groupby('owner_id')['video_id'].transform('count')
    video_data['owner_mean_duration'] = video_data.groupby('owner_id')['duration'].transform('mean')
    video_data['owner_min_duration'] = video_data.groupby('owner_id')['duration'].transform('min')
    video_data['owner_max_duration'] = video_data.groupby('owner_id')['duration'].transform('max')
    video_data['owner_std_duration'] = video_data.groupby('owner_id')['duration'].transform('std')
    video_data['video_hour_after_create'] = (video_data['upload_timestamp'] - video_data['create_date']) / 3600
    video_data['video_hour_after_last_active'] = (video_data['upload_timestamp'] - video_data['last_active_date']) / 3600
    
    video_data['owner_like_sum'] = video_data.groupby('owner_id')['sum'].transform('sum')
    video_data['owner_like_mean'] = video_data.groupby('owner_id')['sum'].transform('mean')
    video_data['owner_like_max'] = video_data.groupby('owner_id')['sum'].transform('max')
    video_data['owner_like_min'] = video_data.groupby('owner_id')['sum'].transform('min')
    video_data['owner_like_std'] = video_data.groupby('owner_id')['sum'].transform('std')

    video_data['owner_count_sum'] = video_data.groupby('owner_id')['count'].transform('sum')
    video_data['owner_count_mean'] = video_data.groupby('owner_id')['count'].transform('mean')
    video_data['owner_count_max'] = video_data.groupby('owner_id')['count'].transform('max')
    video_data['owner_count_min'] = video_data.groupby('owner_id')['count'].transform('min')
    video_data['owner_count_std'] = video_data.groupby('owner_id')['count'].transform('std')
    
    video_data['ow_duration'] = video_data['duration'] / video_data['owner_mean_duration']
    return video_data

In [11]:
def generate_user_features(user_data):
    user_data['count_city_age'] = user_data.groupby('city_id')['age'].transform('count')
    user_data['mean_city_age'] = user_data.groupby('city_id')['age'].transform('mean')
    user_data['otn_city_age_user_age'] = user_data['age'] / user_data['mean_city_age']
    return user_data
user_data = generate_user_features(user_data)

In [12]:
user_data = user_data.fillna(-1)
video_data = generate_video_owner_features(video_data)

In [13]:
df = df.merge(user_data)
df = df.merge(video_data,on='video_id')
df = df.sort_values(by='user_id')
df['user_id'] = df['user_id'].astype(int)
df.columns

Index(['ranks_als', 'rank_id_als', 'ranks_tf', 'rank_id_tf', 'video_id',
       'user_id', 'gender', 'age', 'language', 'city_id_x', 'birth_city_id',
       'create_date_x', 'count_city_age', 'mean_city_age',
       'otn_city_age_user_age', 'owner_id', 'duration', 'upload_timestamp',
       'subscribers_count', 'last_active_date', 'city_id_y', 'create_date_y',
       'sum', 'mean', 'count', 'owner_count', 'owner_mean_duration',
       'owner_min_duration', 'owner_max_duration', 'owner_std_duration',
       'video_hour_after_create', 'video_hour_after_last_active',
       'owner_like_sum', 'owner_like_mean', 'owner_like_max', 'owner_like_min',
       'owner_like_std', 'owner_count_sum', 'owner_count_mean',
       'owner_count_max', 'owner_count_min', 'owner_count_std', 'ow_duration'],
      dtype='object')

In [14]:
def get_df_features(df): 
    df['mean_age_video'] = df.groupby('video_id')['age'].transform('mean')
    df['std_age_video'] = df.groupby('video_id')['age'].transform('std')
    df['birth_city_not_none'] = df['birth_city_id'].apply(lambda x: int(x == -1))
    df['city_not_none'] = df['city_id_x'].apply(lambda x: int(x == -1))
    df['city_rav1'] = df.progress_apply(lambda x: int(x.city_id_x == x.city_id_y and x.city_id_x != -1),axis=1)
    df['city_rav2'] = df.progress_apply(lambda x: int(x.birth_city_id == x.city_id_y and x.city_id_x != -1),axis=1)
    df['city_rav3'] = df.progress_apply(lambda x: int(x.city_id_x == x.birth_city_id and x.city_id_x != -1),axis=1)
    return df

In [15]:
df

ranks_als  rank_id_als  ranks_tf  rank_id_tf  video_id  user_id  \
0          0.026183          0.0 -1.000000       500.0   77671.0        0   
3968517    0.002939         98.0 -1.000000       500.0  129015.0        0   
3948256    0.002954         97.0 -1.000000       500.0  217925.0        0   
3899361    0.002966         96.0 -1.000000       500.0  222069.0        0   
3875172    0.002966         95.0 -1.000000       500.0    8790.0        0   
...             ...          ...       ...         ...       ...      ...   
6936032    0.014922         91.0 -1.000000       500.0   96522.0   152909   
6123628    0.033910         21.0 -1.000000       500.0   12464.0   152909   
3899360    0.014422         97.0 -1.000000       500.0    8790.0   152909   
11044315   0.014462         96.0 -1.000000       500.0  178083.0   152909   
15293504  -1.000000        500.0  1.360769        53.0  177785.0   152909   

         gender   age language  \
0             F  69.0       ru   
3968517       F  69.0       ru   
3948256       F  69.0       ru   
3899361       F  69.0       ru   
3875172       F  69.0       ru   
...         ...   ...      ...   
6936032       M  77.0       ru   
6123628       M  77.0       ru   
3899360       M  77.0       ru   
11044315      M  77.0       ru   
15293504      M  77.0       ru   

                                                  city_id_x  ...  \
0         da8c02ad0ee094bb0e6c69ad8abda0ac11800dce1b4ac5...  ...   
3968517   da8c02ad0ee094bb0e6c69ad8abda0ac11800dce1b4ac5...  ...   
3948256   da8c02ad0ee094bb0e6c69ad8abda0ac11800dce1b4ac5...  ...   
3899361   da8c02ad0ee094bb0e6c69ad8abda0ac11800dce1b4ac5...  ...   
3875172   da8c02ad0ee094bb0e6c69ad8abda0ac11800dce1b4ac5...  ...   
...                                                     ...  ...   
6936032   e88c4137b0344ee336bca089bcd1e2fbf8378053dd735a...  ...   
6123628   e88c4137b0344ee336bca089bcd1e2fbf8378053dd735a...  ...   
3899360   e88c4137b0344ee336bca089bcd1e2fbf8378053dd735a...  ...   
11044315  e88c4137b0344ee336bca089bcd1e2fbf8378053dd735a...  ...   
15293504  e88c4137b0344ee336bca089bcd1e2fbf8378053dd735a...  ...   

         owner_like_mean owner_like_max  owner_like_min  owner_like_std  \
0              36.731884            736               0      104.690196   
3968517        44.473684            832               0      190.719506   
3948256        14.763441           1250               0      129.552860   
3899361         2.510638            144               0       15.019518   
3875172        49.711864           1488               0      118.084507   
...                  ...            ...             ...             ...   
6936032        28.081081           1368               0      134.419455   
6123628       374.318182           3016               0      774.078524   
3899360        49.711864           1488               0      118.084507   
11044315       33.090090            726               0       75.062400   
15293504        5.244898             69               0        9.847865   

          owner_count_sum  owner_count_mean  owner_count_max  owner_count_min  \
0                   45401        328.992754             7895                1   
3968517              1980        104.210526             1845                1   
3948256              3317         35.666667             2770                1   
3899361              4947         52.627660             4041                1   
3875172             71486        201.937853             2667                1   
...                   ...               ...              ...              ...   
6936032             31741        107.233108             5831                1   
6123628             20209        918.590909             7084                1   
3899360             71486        201.937853             2667                1   
11044315           147913        166.568694             2752                1   
15293504             1370         27.959184              216       

In [32]:
#df = df.merge(video_like_faeatures.reset_index())

In [16]:
from sklearn.model_selection import GroupShuffleSplit 

splitter = GroupShuffleSplit(test_size=0.20, n_splits=2, random_state=56)
split = splitter.split(df, groups=df['user_id'])
train_inds, test_inds = next(split)

train_df = df.iloc[train_inds]
val_df = df.iloc[test_inds]

In [17]:
del df
del val_no_targets
#del train
del train_inds
del test_inds

In [18]:
val_df = val_df.sort_values(by='user_id')
train_df = train_df.sort_values(by='user_id')
train_df['user_id'] = train_df['user_id'].astype(int)
val_df['user_id'] = val_df['user_id'].astype(int)

In [ ]:
val_df

In [19]:
val_df.to_csv('val_ans.csv', index=False)

In [20]:
train_df.to_csv('train_ans.csv', index=False)

inference

In [2]:
val_df = pd.read_csv('val_ans.csv')
train_df = pd.read_csv('train_ans.csv')

In [22]:
train_df['label'] = train_df['label'].replace({0.0:0, 0.1:1, 1:10})
val_df['label'] = val_df['label'].replace({0.0:0, 0.1:1, 1:10})

train_df['label'] = train_df['label'].astype(int)
val_df['label'] = val_df['label'].astype(int)

KeyError: 'label'

In [39]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_df['gender'] = le.fit_transform(train_df['gender'].values.tolist())
train_df['language'] = le.fit_transform(train_df['language'].values.tolist())

val_df['gender'] = le.fit_transform(val_df['gender'].values.tolist())
val_df['language'] = le.fit_transform(val_df['language'].values.tolist())

In [40]:
X_train = train_df.drop(['label','user_id_per_video_id','city_id_x','city_id_y','birth_city_id','create_date_x'],axis=1)
y_train = train_df['label']
train_group = train_df.groupby("user_id")["user_id"].count().to_numpy()

X_test = val_df.drop(['label','user_id_per_video_id','city_id_x','city_id_y','birth_city_id','create_date_x'],axis=1)
y_test = val_df['label']
test_group = val_df.groupby("user_id")["user_id"].count().to_numpy()

In [41]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(len(train_group), len(test_group))
print(sum(train_group), sum(test_group))
print(max(train_group), max(test_group))

(12611533, 39) (12611533,)
(3153161, 39) (3153161,)
80000 20000
12611533 3153161
200 200


In [42]:
import lightgbm
dataset_train = lightgbm.Dataset(
    data=X_train, 
    label=y_train, 
    group=train_group
    )

dataset_val = lightgbm.Dataset(
    data=X_test, 
    label=y_test, 
    group=test_group,
    reference=dataset_train, 

    )

In [43]:
del train_df
del val_df

In [44]:
params = {
    "objective": "lambdarank",
    "metric": ["map", "auc"],
    "eval_at": 20,
    "boosting_type": "gbdt",
    "is_unbalance": True,
    "learning_rate": 0.05,
    'lambda_l1': 0.05,
    'lambda_l2': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'force_col_wise': True
    }



In [45]:
ranker = lightgbm.train(
    params=params,
    train_set=dataset_train,
    num_boost_round=10_000,
    valid_sets=[dataset_val],
    # feval=custom_recall
    callbacks=[
        lightgbm.early_stopping(100),
        lightgbm.log_evaluation(50)
    ],
)

[LightGBM] [Info] Total groups: 80000, total data: 12611533
[LightGBM] [Info] Total Bins 7788
[LightGBM] [Info] Number of data points in the train set: 12611533, number of used features: 39
[LightGBM] [Info] Total groups: 20000, total data: 3153161
Training until validation scores don't improve for 100 rounds
[50]	valid_0's map@20: 0.260926	valid_0's auc: 0.756199
[100]	valid_0's map@20: 0.26471	valid_0's auc: 0.764442
[150]	valid_0's map@20: 0.26674	valid_0's auc: 0.768549
[200]	valid_0's map@20: 0.267888	valid_0's auc: 0.770838
[250]	valid_0's map@20: 0.268897	valid_0's auc: 0.772563
[300]	valid_0's map@20: 0.26878	valid_0's auc: 0.773589
[350]	valid_0's map@20: 0.269085	valid_0's auc: 0.774219
[400]	valid_0's map@20: 0.269269	valid_0's auc: 0.774421
[450]	valid_0's map@20: 0.269372	valid_0's auc: 0.774913
[500]	valid_0's map@20: 0.269305	valid_0's auc: 0.775437
Early stopping, best iteration is:
[425]	valid_0's map@20: 0.269509	valid_0's auc: 0.774724


In [46]:
ranker.save_model('model_lgbm.txt', num_iteration=ranker.best_iteration)

In [47]:
import joblib
# save model
joblib.dump(ranker, 'lgb.pkl')
# load model
#gbm_pickle = joblib.load('lgb.pkl')

['lgb.pkl']

In [ ]:
#### INFERENSE

In [47]:
test_predicts_tf = pd.read_parquet("TFIDF_preds_test100.parquet")
test_predicts_als = pd.read_parquet("BM25_preds_test100.parquet")


In [48]:
sample_sub = pd.read_csv('lol.csv')['user_id']
test_predicts_tf['user_id'] = sample_sub.values
test_predicts_tf['rank_i'] = test_predicts_tf['preds'].map(lambda x: list(range(len(x))))
test_predicts_als['user_id'] = sample_sub.values
test_predicts_als['rank_i'] = test_predicts_als['preds'].map(lambda x: list(range(len(x))))

In [49]:
df_tf = create_data(test_predicts_tf)
df_als = create_data(test_predicts_als)

  0%|          | 0/152911 [00:00<?, ?it/s]

  0%|          | 0/152911 [00:00<?, ?it/s]

In [50]:
df_tf = df_tf.rename({'ranks':'ranks_tf','rank_id':'rank_id_tf'},axis=1)
df_als = df_als.rename({'ranks':'ranks_als','rank_id':'rank_id_als'},axis=1)

In [51]:
tqdm.pandas()
df_tf['user_id_per_video_id'] = df_tf.progress_apply(lambda x: str(x.video_id) + '_' + str(x.user_id),axis=1)
df_als['user_id_per_video_id'] = df_als.progress_apply(lambda x: str(x.video_id) + '_' + str(x.user_id),axis=1)

  0%|          | 0/15287689 [00:00<?, ?it/s]

  0%|          | 0/15291100 [00:00<?, ?it/s]

In [52]:
df_als = df_als.set_index('user_id_per_video_id')
df_tf = df_tf.set_index('user_id_per_video_id')

In [2]:
df_all = pd.concat([df_als,df_tf],axis=1)
df_all['video_id'] = df_all['video_id'].fillna(-1)
video_id = np.max(df_all['video_id'].values,axis=-1)
df_all['user_id'] = df_all['user_id'].fillna(-1)
users_id = np.max(df_all['user_id'].values,axis=-1)
df_all = df_all.drop(['user_id','video_id'],axis=1)
df_all['video_id'] = video_id
df_all['user_id'] = users_id
df = df_all
df['rank_id_als'] = df['rank_id_als'].fillna(500)
df['rank_id_tf'] = df['rank_id_tf'].fillna(500)
df['ranks_als'] = df['ranks_als'].fillna(-1)
df['ranks_tf'] = df['ranks_tf'].fillna(-1)
df.to_parquet('DF_TEST.parquet.gzip',compression='gzip')

NameError: name 'pd' is not defined

In [54]:
del df_all

In [ ]:
df = pd.read_parquet('DF_TEST.parquet.gzip')

In [1]:
df = df.merge(user_data)
df = df.merge(video_data,on='video_id')

NameError: name 'df' is not defined

In [ ]:
df = df.sort_values(by='user_id')

In [ ]:
df['user_id'] = df['user_id'].astype(int)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['gender'] = le.fit_transform(df['gender'].values.tolist())
df['language'] = le.fit_transform(df['language'].values.tolist())

In [ ]:
df.columns

In [ ]:
X_train = train_df.drop(['label','city_id'],axis=1)
y_train = train_df['label']
train_group = train_df.groupby("user_id")["user_id"].count().to_numpy()

In [66]:
df.columns()

TypeError: 'Index' object is not callable

In [87]:
df['scores'] = cbm.predict(predict_pool)

In [88]:
steps = df.groupby('user_id')['ranks'].agg('count').sort_index()

In [ ]:
df

In [89]:
steps = steps.tolist()

In [90]:
model_ranks,als_ranks = [],[]
start = 0
for step in tqdm(steps):
    df_ = df.iloc[start:start+step]
    start += step
    als_ranks += [df_.sort_values(by='scores')['video_id'][::-1][:10].tolist()]
    model_ranks += [df_.sort_values(by='scores')['video_id'][::-1][:10].tolist()]

  0%|          | 0/152911 [00:00<?, ?it/s]

In [91]:
sample_sub = pd.read_csv('lol.csv')

In [92]:
sample_sub = sample_sub.sort_values(by='user_id')

In [93]:
predicted_cols=[' '.join(map(str, i)) for i in model_ranks]

In [94]:
sample_sub['recommendation'] = predicted_cols

In [53]:
sample_sub.to_csv('Simple_CatBoostR79.csv',index=False)

In [95]:
sample_sub['count_tr'] = sample_sub['recommendation'].map(lambda x: len(x.split()))

In [96]:
sample_sub[sample_sub['count_tr'] != 10]

user_id                                   recommendation  count_tr
48959     20954  242750 195428 169983 239035 35632 103098 229566         7
98527     22025          74601 143427 16701 180173 222751 252568         6
144672    55148    141781 79672 47517 251129 137219 24127 149457         7
98927     78973                                     120362 29784         2

In [97]:
recs = sample_sub['recommendation']
recs[98527] = '24671 65611 45311 132648 217384 34204 166282 51799 131005 188589'
recs[98927] = '24671 65611 45311 132648 217384 34204 166282 51799 131005 188589'
recs[144672] = '24671 65611 45311 132648 217384 34204 166282 51799 131005 188589'
recs[48959] = '24671 65611 45311 132648 217384 34204 166282 51799 131005 188589'

/tmp/ipykernel_53370/1353744486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recs[98527] = '24671 65611 45311 132648 217384 34204 166282 51799 131005 188589'
/tmp/ipykernel_53370/1353744486.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recs[98927] = '24671 65611 45311 132648 217384 34204 166282 51799 131005 188589'
/tmp/ipykernel_53370/1353744486.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recs[144672] = '24671 656

In [40]:
sample_sub['recommendation'] = recs

In [43]:
sample_sub['count_tr'].mean()

10.0

In [84]:
sample_socres = pd.read_csv('Simple_catbooost.csv')
sample_socres

FileNotFoundError: [Errno 2] No such file or directory: 'Simple_catbooost.csv'

In [98]:
sample_sub[['user_id','recommendation']].to_csv('Simple_catboost_bm82.csv',index=False)

In [ ]:
sample_sub